In [13]:
!pip install mlx-whisper

  Using cached mlx_whisper-0.4.1-py3-none-any.whl (782 kB)
  Using cached mlx-0.20.0-cp311-cp311-macosx_14_0_arm64.whl (23.5 MB)
  Using cached numba-0.60.0-cp311-cp311-macosx_11_0_arm64.whl (2.6 MB)
  Using cached numpy-2.1.3-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Using cached torch-2.5.1-cp311-none-macosx_11_0_arm64.whl (63.9 MB)
  Using cached tqdm-4.67.0-py3-none-any.whl (78 kB)
  Using cached more_itertools-10.5.0-py3-none-any.whl (60 kB)
  Using cached tiktoken-0.8.0-cp311-cp311-macosx_11_0_arm64.whl (982 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl (447 kB)
  Using cached scipy-1.14.1-cp311-cp311-macosx_14_0_arm64.whl (23.1 MB)
  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl (172 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached llvmlite-0.43.0-cp311-cp311-macosx_11_0_arm64.whl (28.8 MB)
  Using cached numpy-

In [1]:
import mlx_whisper

speech_file = "./data/vlog_rotter_audio.m4a"
text = mlx_whisper.transcribe(speech_file, path_or_hf_repo="mlx-community/whisper-large-v3-mlx")

text


/Users/byby/DevProjects/fcpx-edit-buddy/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 37701.61it/s]


{'text': " bonjour les amis enfin moment que je dois vous parler vous raconter un peu mais péripétiez en regardant et je veux enfin prendre le temps de le faire alors je vais parler en trois étapes je vais faire une vidéo en trois étapes la première ce sera un peu par rapport à mon arrivée mon accueil et tout ça ensuite je vous dis un peu comment ça s'est passé recherche de logement et de taf ici et enfin je vais un peu plus vous parler de mon état mental parce que je pense que c'est la chose que je manage le plus actuellement c'est parti alors avant de parler de l'arrivée et de l'accueil je vous parlez un peu du contexte faut savoir que je pars de paris sans boulot sans logement non plus parce que j'ai envie de vivre à rotterdam parce que encore une fois je suis allé à rotterdam c'est une ville qui m'a tapé dans l'oeil et j'ai eu envie de faire une expérience là bas sauf que je n'ai pas voulu attendre que toutes les conditions soient réunies avant de partir je me suis dit on va y alle

In [32]:
def create_srt_from_transcription(transcription_dict, output_file="output.srt"):
    """
    Convert a transcription dictionary to SRT format and save to file.
    
    Args:
        transcription_dict (dict): Dictionary containing transcription data
        output_file (str): Path to save the SRT file
    """
    def format_timestamp(seconds):
        """Convert seconds to SRT timestamp format: HH:MM:SS,mmm"""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        milliseconds = int(round((seconds - int(seconds)) * 1000))
        return f"{hours:02d}:{minutes:02d}:{secs:02d},{milliseconds:03d}"

    # Extract and sort segments
    segments = transcription_dict.get('segments', [])
    segments.sort(key=lambda x: x.get('start', 0))

    # Process segments to fix overlaps
    processed_segments = []
    previous_end = 0
    
    for segment in segments:
        start_time = segment.get('start', 0)
        end_time = segment.get('end', 0)
        
        # Adjust start_time if it overlaps with previous segment
        if start_time <= previous_end:
            start_time = previous_end + 0.001  # Add 1ms gap
        
        # Ensure end_time is after start_time
        if end_time <= start_time:
            end_time = start_time + 0.001  # Minimum duration of 1ms
            
        # Update previous_end for next iteration
        previous_end = end_time
        
        processed_segments.append({
            'start': start_time,
            'end': end_time,
            'text': segment.get('text', '').strip()
        })

    # Write to file
    with open(output_file, 'w', encoding='utf-8') as f:
        empty=0
        for i, segment in enumerate(processed_segments, 1):
            text = segment['text']
            if not text:  # Skip empty segments
                empty+=1
                continue
                
            srt_segment = (
                f"{i-empty}\n"
                f"{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}\n"
                f"{text}\n\n"
            )
            f.write(srt_segment)
    
    print(f"SRT file '{output_file}' created successfully.")


speech_file = "./data/p1_tiktok.m4a"
transcription = mlx_whisper.transcribe(speech_file, path_or_hf_repo="mlx-community/whisper-large-v3-mlx", initial_prompt="La visualisation ")
create_srt_from_transcription(transcription, output_file="p1_tiktok.srt")

SRT file 'p1_tiktok.srt' created successfully.


In [21]:
import srt
from datetime import timedelta

def check_caption_duration(file_path):
    with open(file_path, 'r') as f:
        srt_data = f.read()

    subtitles = list(srt.parse(srt_data))

    for subtitle in subtitles:
        caption_duration = subtitle.end - subtitle.start
        if caption_duration.total_seconds() < 0.002:  # 2 milliseconds
            print(f"Caption {subtitle.index} is less than 2 milliseconds.")

# Replace 'subtitle_file.srt' with the path to your SRT file
srt_file_path = 'vlogrotter.srt'

check_caption_duration(srt_file_path)

Caption 115 is less than 2 milliseconds.


In [18]:
help(mlx_whisper.transcribe)

Help on function transcribe in module mlx_whisper.transcribe:

transcribe(audio: Union[str, numpy.ndarray, mlx.core.array], *, path_or_hf_repo: str = 'mlx-community/whisper-tiny', verbose: Optional[bool] = None, temperature: Union[float, Tuple[float, ...]] = (0.0, 0.2, 0.4, 0.6, 0.8, 1.0), compression_ratio_threshold: Optional[float] = 2.4, logprob_threshold: Optional[float] = -1.0, no_speech_threshold: Optional[float] = 0.6, condition_on_previous_text: bool = True, initial_prompt: Optional[str] = None, word_timestamps: bool = False, prepend_punctuations: str = '"\'“¿([{-', append_punctuations: str = '"\'.。,，!！?？:：”)]}、', clip_timestamps: Union[str, List[float]] = '0', hallucination_silence_threshold: Optional[float] = None, **decode_options)
    Transcribe an audio file using Whisper
    
    Parameters
    ----------
    audio: Union[str, np.ndarray, mx.array]
        The path to the audio file to open, or the audio waveform
    
    path_or_hf_repo: str
        The localpath to the 

In [2]:
import xml.etree.ElementTree as ET
import pysrt  # To parse .srt files

# Load the .srt file and extract timestamps and subtitles
def parse_srt(srt_path):
    subtitles = pysrt.open(srt_path)
    segments = []
    for subtitle in subtitles:
        start = (subtitle.start.hours * 3600) + (subtitle.start.minutes * 60) + subtitle.start.seconds
        end = (subtitle.end.hours * 3600) + (subtitle.end.minutes * 60) + subtitle.end.seconds
        duration = end - start
        segments.append({
            'start': start,
            'end': end,
            'duration': duration,
            'text': subtitle.text
        })
    return segments

# Load the .fcpxml file, parse it, and keep <resources> intact
def load_fcpxml(fcpxml_path):
    tree = ET.parse(fcpxml_path)
    root = tree.getroot()
    return tree, root

# Insert subtitles as captions in .fcpxml based on segments from .srt
def add_captions_to_fcpxml(root, segments):
    # Locate the timeline where we will insert clips and captions
    timeline = root.find(".//sequence")  # Adjust this path if needed based on .fcpxml structure

    for segment in segments:
        # Create a new clip item for each segment
        asset_clip = ET.SubElement(timeline, "asset-clip")
        asset_clip.set("start", str(segment['start']))
        asset_clip.set("duration", str(segment['duration']))
        asset_clip.set("name", "Segment: " + segment['text'][:30])  # Clip name truncated for brevity

        # Add text element to the clip for captions
        text = ET.SubElement(asset_clip, "text")
        text.text = segment['text']

# Save modified .fcpxml to a new file
def save_fcpxml(tree, output_path):
    tree.write(output_path, encoding="utf-8", xml_declaration=True)

# Main script execution
srt_path = "output.srt"
fcpxml_path = "actual.fcpxml"
output_fcpxml_path = "test.fcpxml"

# Parse SRT and FCPXML
segments = parse_srt(srt_path)
tree, root = load_fcpxml(fcpxml_path)

# Add captions and colors based on parsed SRT segments
add_captions_to_fcpxml(root, segments)

# Save the modified FCPXML
save_fcpxml(tree, output_fcpxml_path)

print(f"Modified .fcpxml saved as: {output_fcpxml_path}")

ModuleNotFoundError: No module named 'pysrt'

In [12]:
fcpxml_path = "actual.fcpxml"
tree, root = load_fcpxml(fcpxml_path)


[('version', '1.12')]

In [17]:
import xml.etree.ElementTree as ET
import pysrt
from datetime import timedelta

def parse_srt(srt_path):
    subtitles = pysrt.open(srt_path)
    segments = []
    for subtitle in subtitles:
        start = (subtitle.start.hours * 3600) + (subtitle.start.minutes * 60) + subtitle.start.seconds
        end = (subtitle.end.hours * 3600) + (subtitle.end.minutes * 60) + subtitle.end.seconds
        duration = end - start
        segments.append({
            'start': start,
            'end': end,
            'duration': duration,
            'text': subtitle.text
        })
    return segments

def format_duration(seconds):
    return f"{int(seconds * 1000)}/3000s"

def create_caption_element(caption_id, subtitle, ts_id):
    caption = ET.Element('caption', {
        'lane': "1",
        'offset': "0s",
        'name': subtitle['text'],
        'start': f"{subtitle['start']}s",
        'duration': format_duration(subtitle['duration']),
        'role': "SRT?captionFormat=SRT.fr-FR"
    })
    
    text = ET.SubElement(caption, 'text', {'placement': "bottom"})
    text_style = ET.SubElement(text, 'text-style', {'ref': ts_id})
    text_style.text = subtitle['text']
    
    text_style_def = ET.SubElement(caption, 'text-style-def', {'id': ts_id})
    ET.SubElement(text_style_def, 'text-style', {
        'font': ".AppleSystemUIFont",
        'fontSize': "10",
        'fontFace': "Regular",
        'fontColor': "1 0.843 0 1",
        'backgroundColor': "0 0 0 0"
    })
    
    return caption

def create_asset_clip_element(clip_id, subtitle, ts_id):
    asset_clip = ET.Element('asset-clip', {
        'ref': "r2",
        'offset': "0s",
        'name': f"Subtitle Clip {clip_id}",
        'start': f"{subtitle['start']}s",
        'duration': format_duration(subtitle['duration']),
        'format': "r1",
        'tcFormat': "NDF",
        'audioRole': "dialogue"
    })
    
    ET.SubElement(asset_clip, 'conform-rate', {'scaleEnabled': "0"})
    
    caption = create_caption_element(clip_id, subtitle, f"ts{clip_id}")
    asset_clip.append(caption)
    
    return asset_clip

def generate_fcpxml(srt_file, output_xml):
    subtitles = parse_srt(srt_file)
    
    fcpxml = ET.Element('fcpxml', {'version': "1.10"})
    resources = ET.SubElement(fcpxml, 'resources')
    
    library = ET.SubElement(fcpxml, 'library', {'location': "file:///Users/byby/Movies/Bybysker%20TV.fcpbundle/"})
    event = ET.SubElement(library, 'event', {'name': "youtube", 'uid': "D3A94D22-5900-4F79-BF7E-FEEF9E5EE92D"})
    project = ET.SubElement(event, 'project', {'name': "vlog_rotter", 'uid': "419AF4A1-84A0-4B0F-891E-354234E5DA57", 'modDate': "2024-11-11 18:34:39 +0100"})
    sequence = ET.SubElement(project, 'sequence', {
        'format': "r1",
        'duration': "3142200/3000s",
        'tcStart': "0s",
        'tcFormat': "NDF",
        'audioLayout': "stereo",
        'audioRate': "48k"
    })
    spine = ET.SubElement(sequence, 'spine')
    
    for idx, subtitle in enumerate(subtitles, start=1):
        asset_clip = create_asset_clip_element(idx, subtitle, f"ts{idx}")
        spine.append(asset_clip)
    
    tree = ET.ElementTree(fcpxml)
    tree.write(output_xml, encoding='utf-8', xml_declaration=True)

if __name__ == "__main__":
    srt_input = "output.srt"
    xml_output = "generated_output.fcpxml"
    generate_fcpxml(srt_input, xml_output)

In [20]:
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom
import pysrt

def parse_srt(srt_path):
    subtitles = pysrt.open(srt_path)
    segments = []
    for subtitle in subtitles:
        start = (subtitle.start.hours * 3600) + (subtitle.start.minutes * 60) + subtitle.start.seconds
        end = (subtitle.end.hours * 3600) + (subtitle.end.minutes * 60) + subtitle.end.seconds
        duration = end - start
        segments.append({
            'start': start,
            'end': end,
            'duration': duration,
            'text': subtitle.text
        })
    return segments

def format_duration(seconds):
    return f"{int(seconds * 1000)}/3000s"

def create_caption_element(caption_id, subtitle, ts_id):
    caption = ET.Element('caption', {
        'lane': "1",
        'offset': "0s",
        'name': subtitle['text'],
        'start': f"{subtitle['start']}s",
        'duration': format_duration(subtitle['duration']),
        'role': "SRT?captionFormat=SRT.fr-FR"
    })
    
    text = ET.SubElement(caption, 'text', {'placement': "bottom"})
    text_style = ET.SubElement(text, 'text-style', {'ref': ts_id})
    text_style.text = subtitle['text']
    
    text_style_def = ET.SubElement(caption, 'text-style-def', {'id': ts_id})
    ET.SubElement(text_style_def, 'text-style', {
        'font': ".AppleSystemUIFont",
        'fontSize': "10",
        'fontFace': "Regular",
        'fontColor': "1 0.843 0 1",
        'backgroundColor': "0 0 0 0"
    })
    
    return caption

def create_asset_clip_element(clip_id, subtitle, ts_id):
    asset_clip = ET.Element('asset-clip', {
        'ref': "r2",
        'offset': "0s",
        'name': f"Subtitle Clip {clip_id}",
        'start': f"{subtitle['start']}s",
        'duration': format_duration(subtitle['duration']),
        'format': "r1",
        'tcFormat': "NDF",
        'audioRole': "dialogue"
    })
    
    ET.SubElement(asset_clip, 'conform-rate', {'scaleEnabled': "0"})
    
    caption = create_caption_element(clip_id, subtitle, f"ts{clip_id}")
    asset_clip.append(caption)
    
    return asset_clip

def prettify_xml(elem):
    """Return a pretty-printed XML string for the Element."""
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def generate_fcpxml(srt_file, output_xml):
    subtitles = parse_srt(srt_file)
    
    fcpxml = ET.Element('fcpxml', {'version': "1.10"})
    resources = ET.SubElement(fcpxml, 'resources')
    
    library = ET.SubElement(fcpxml, 'library', {'location': "file:///Users/byby/Movies/Bybysker%20TV.fcpbundle/"})
    event = ET.SubElement(library, 'event', {'name': "youtube", 'uid': "D3A94D22-5900-4F79-BF7E-FEEF9E5EE92D"})
    project = ET.SubElement(event, 'project', {'name': "vlog_rotter", 'uid': "419AF4A1-84A0-4B0F-891E-354234E5DA57", 'modDate': "2024-11-11 18:34:39 +0100"})
    sequence = ET.SubElement(project, 'sequence', {
        'format': "r1",
        'duration': "3142200/3000s",
        'tcStart': "0s",
        'tcFormat': "NDF",
        'audioLayout': "stereo",
        'audioRate': "48k"
    })
    spine = ET.SubElement(sequence, 'spine')
    
    for idx, subtitle in enumerate(subtitles, start=1):
        asset_clip = create_asset_clip_element(idx, subtitle, f"ts{idx}")
        spine.append(asset_clip)
    
    pretty_xml = prettify_xml(fcpxml)
    with open(output_xml, 'w', encoding='utf-8') as f:
        f.write(pretty_xml)

if __name__ == "__main__":
    srt_input = "output.srt"
    xml_output = "generated_output.fcpxml"
    generate_fcpxml(srt_input, xml_output)

In [12]:
import os 
import json
from dotenv import load_dotenv
from urllib import parse, request

load_dotenv()

def get_n_gifs(query, n=5):
    search_url = "http://api.giphy.com/v1/gifs/search"

    params = parse.urlencode({
      "q": query,
      "api_key": os.getenv("GIPHY_API_KEY"),
      "limit": "5"
    })

    with request.urlopen("".join((search_url, "?", params))) as response:
      data = json.loads(response.read())

    return [ gif['images']['original']['url'] for gif in data['data']]


query = "cat"
gifs_urls = get_n_gifs(query)

gifs_urls

['https://media0.giphy.com/media/l0ExdMHUDKteztyfe/giphy.gif?cid=86377d4adczcz3e2s5ozvaolfhxwtwujoji9dobz5kmdql9v&ep=v1_gifs_search&rid=giphy.gif&ct=g',
 'https://media1.giphy.com/media/CjmvTCZf2U3p09Cn0h/giphy.gif?cid=86377d4adczcz3e2s5ozvaolfhxwtwujoji9dobz5kmdql9v&ep=v1_gifs_search&rid=giphy.gif&ct=g',
 'https://media0.giphy.com/media/l4KibK3JwaVo0CjDO/giphy.gif?cid=86377d4adczcz3e2s5ozvaolfhxwtwujoji9dobz5kmdql9v&ep=v1_gifs_search&rid=giphy.gif&ct=g',
 'https://media1.giphy.com/media/MDJ9IbxxvDUQM/giphy.gif?cid=86377d4adczcz3e2s5ozvaolfhxwtwujoji9dobz5kmdql9v&ep=v1_gifs_search&rid=giphy.gif&ct=g',
 'https://media3.giphy.com/media/mlvseq9yvZhba/giphy.gif?cid=86377d4adczcz3e2s5ozvaolfhxwtwujoji9dobz5kmdql9v&ep=v1_gifs_search&rid=giphy.gif&ct=g']